In [1]:
import tweepy
import pandas as pd

## Read the Bearer Token for Authentication

In [2]:
with open("./bearer_token.txt") as file:
    BEARER_TOKEN = file.read()

## Connect to the Twitter Client

In [3]:
client = tweepy.Client(bearer_token=BEARER_TOKEN)

## Read Tweets
This will retrieve the most recent 100 tweets written in english that contain the hashtags "#iphone" or "#apple" for the past week. The query can only be 512 characters long.

In [4]:
# Replace with your own search query
query = '(#iphone OR #Apple) lang:en has:hashtags -is:reply'

tweets = client.search_recent_tweets(
    query=query,
    tweet_fields=['created_at', 'possibly_sensitive', 'public_metrics'],
    max_results=100
)

## Convert the Tweets to a Pandas DataFram

In [5]:
raw_tweets = [
    [
        t.id,
        t.text,
        t.created_at,
        t.possibly_sensitive,
        t.public_metrics['retweet_count'],
        t.public_metrics['reply_count'],
        t.public_metrics['like_count'],
        t.public_metrics['quote_count'],
    ]
    for t in tweets.data
]

df = pd.DataFrame(raw_tweets, columns=[
    "id", "text", "created_at", "is_sensitive", "retweet_count",
    "reply_count", "like_count", "quote_count"
])

### Preview the DataFrame

In [6]:
df

,id,text,created_at,is_sensitive,retweet_count,reply_count,like_count,quote_count
0,1593001348341338113,RT @appleinsider: How To Create and Share Cale...,2022-11-16 22:01:32+00:00,False,1,0,0,0
1,1593001333212483585,i’d like ask something. how do i snyc all musi...,2022-11-16 22:01:29+00:00,False,0,0,0,0
2,1593001286173347841,RT @GiveawayBase: Enter for a #chance to #win ...,2022-11-16 22:01:17+00:00,False,2,0,0,0
3,1593001267131187203,How To Create and Share Calendars in iOS 16 ht...,2022-11-16 22:01:13+00:00,False,1,0,2,0
4,1593001231110152192,Abstract Flowers Pattern iPhone Case\nlink :ht...,2022-11-16 22:01:04+00:00,False,0,0,0,0
...,...,...,...,...,...,...,...,...
95,1592992946013732864,RT @OptionsProOI: #Apple $AAPL Unusual #Option...,2022-11-16 21:28:09+00:00,False,2,0,0,0
96,1592992705977946114,Ah yeah. More #iOS16 #bugs. #WellesBugs #Appl...,2022-11-16 21:27:12+00:00,False,0,0,0,0
97,1592992656740601857,"unforgettable” iMovie /apple Starring, filmin...",2022-11-16 21:27:00+00:00,False,0,0,0,0
98,1592992434300284928,RT @jacobincambodia: Captured this lightning s...,2022-11-16 21:26:07+00:00,False,55,0,0,0


### Save the Tweets DataFrame for Testing

In [7]:
df.to_csv("test_data.csv")